In [14]:
from transformers import BartForConditionalGeneration, BartTokenizer
import spacy
import re
import textwrap

# Load BART model
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Function to split text into chunks
def split_text(text, max_chunk_size=1000):
    return textwrap.wrap(text, width=max_chunk_size, break_long_words=False, replace_whitespace=False)

# Extract references to tables, figures, and equations
def extract_references(text):
    ref_pattern = r"(Table|Figure|Equation|Fig|Eq)\s?\d+"
    references = re.findall(ref_pattern, text)
    return list(set(references))  # Remove duplicates

# Summarization function
def summarize_text(text, max_input_length=1024, max_output_length=300):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_input_length, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_output_length, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Convert to bullet points
    bullet_points = summary.split(". ")
    bullet_points = [f"- {point.strip()}." for point in bullet_points if point.strip()]

    return "\n".join(bullet_points)

# Main function
def summarize_research_paper(text):
    chunks = split_text(text, max_chunk_size=800)
    full_summary = ""
    all_references = set()

    for i, chunk in enumerate(chunks):
        print(f"\n🔹 Processing Chunk {i+1}/{len(chunks)}...\n")
        references = extract_references(chunk)
        summary = summarize_text(chunk)

        print(summary)
        print("\n🔷 References Detected:", references)

        full_summary += "\n" + summary
        all_references.update(references)

    return full_summary, all_references

# Example: Research paper text
research_paper = """

"""

# Run summarization
final_summary, detected_references = summarize_research_paper(research_paper)

# Print results
print("\n🔷 Final Summary:\n")
print(final_summary)

if detected_references:
    print("\n📌 References Detected:\n")
    print(", ".join(detected_references))



🔹 Processing Chunk 1/27...

- 4thMediterranean Conference on Embedded ComputingMECO - 2015Budva, Montenegro.
- An Intelligent System for Diabetes Prediction.
- A number of   researches have focused either in using one of the algorithms or in the comparisons of the performances..

🔷 References Detected: []

🔹 Processing Chunk 2/27...

- This paper focuses on the joint implementation of two algorithms on a given dataset.
- The dataset contains some attributes that have not been previously used in computer-based evaluations.
- The results show that the implementation of the algorithms improves the overall reliability of the system.
- This is crucial in the computer -supported diabetes diagnostic process..

🔷 References Detected: []

🔹 Processing Chunk 3/27...

- Diabetes is a chronic disease caused by the increase in blood sugar.
- The number of diabetic patients has increased drastically in recent years.
- Diabetes is the major cause for heart stroke, kidney failure, lower -limb amputat